In [2]:
import pandas as pd
import bokeh.plotting as bpl
import bokeh.models as bmd
import bokeh.layouts as bly
import bokeh.transform as btr
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
from datetime import datetime 
import timeit

In [ ]:
data = pd.read_csv("http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip", compression = "zip", encoding = "latin1")

In [ ]:
pio.templates.default = "plotly_white"
pio.templates
print(np.__version__)

In [ ]:
data 

In [ ]:
last_update = data["FECHA_ACTUALIZACION"].iloc[0]
date_last_update = datetime.strptime(last_update, '%Y-%m-%d')

if date_last_update.day < 10:
    day = "0" + str(date_last_update.day) 
else:
    day = str(date_last_update.day) 

if date_last_update.month < 10:
    month = "0" + str(date_last_update.month)
else:
    month = str(date_last_update.month)

str_last_update = day + "/" + month + "/" + str(date_last_update.year) 
str_last_update

In [ ]:
data[["FECHA_ACTUALIZACION", "FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF"]] #COLUMNA QUE ME INTERESA DataFrame de una columna -----> Serie

In [ ]:
data[['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN']]

In [ ]:
data.loc[10] #INFORMACION DE LA FILA 10

In [ ]:
data.loc[[1,3,6]]

In [ ]:
#Casos negativos, sospechosos y activos
data["CLASIFICACION_FINAL"].value_counts()

In [ ]:
#Casos negativos ---> 7
condicion_negativos = data["CLASIFICACION_FINAL"] == 7

In [ ]:
condicion_negativos.value_counts()

In [ ]:
 negativos = data[condicion_negativos] #Filtrando el data FRAME para casos negativos
 print("Casos Negativos =",len(negativos))

In [ ]:
# 1,2,3 CASOS CONFIRMADOS
confirmados = data[data["CLASIFICACION_FINAL"].isin([1,2,3])]
print("Casos confirmados =",len(confirmados))

In [ ]:
confirmados["CLASIFICACION_FINAL"].value_counts()

In [ ]:
#CASOS SORPECHOSOS 6 --> sospechosos sin muestra", "sospechosos con posibilidad de resultado" y "sospechosos sin posibilidad de resultado".
sospechosos = data[data["CLASIFICACION_FINAL"] == 6]
print("Sospechosos Totales = ", len(sospechosos))

In [ ]:
sospechosos_sin_muestra=data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==97)]
#sospechosos_sin_muestra
print("Sospechosos sin muestra =",len(sospechosos_sin_muestra))

In [ ]:
sospechosos_con_posibilidad =data[(data["CLASIFICACION_FINAL"]==6)&(data["RESULTADO_LAB"]==3)]
#sospechosos_sin_muestra
print("Sospechosos con posibilidad =",len(sospechosos_con_posibilidad))

In [ ]:
sospechosos_sin_posibilidad=data[(data["CLASIFICACION_FINAL"].isin([6,5,4]))&(data["RESULTADO_LAB"].isin([4]))]
print("Sospechosos sin posibilidad =", len(sospechosos_sin_posibilidad))

In [ ]:
#Buscando defunciones 
data.columns

In [ ]:
#data["FECHA_DEF"].value_counts()

In [ ]:
defunciones = confirmados[confirmados["FECHA_DEF"] != "9999-99-99"]
print("Defunciones =", len(defunciones))


In [ ]:
grupos = confirmados.groupby("FECHA_INGRESO")  #agrupando por fecha
confirmados_por_fecha = grupos.size().to_frame("confirmados").reset_index()   #INICIALMENTE grupos.size() es una serie con la fecha como ind
confirmados_por_fecha

In [ ]:
confirmados_por_fecha.dtypes

In [ ]:
confirmados_por_fecha["FECHA_INGRESO"][:3]

In [ ]:
confirmados_por_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")
confirmados_por_fecha

In [ ]:
confirmados_por_fecha.dtypes

In [ ]:
defunciones_por_fecha = defunciones.groupby("FECHA_DEF").size().to_frame("defunciones").reset_index()
defunciones_por_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_fecha


In [ ]:
sospechosos_por_fecha = sospechosos.groupby("FECHA_SINTOMAS").size().to_frame("sospechosos").reset_index()
sospechosos_por_fecha["fecha_sintomas"] = pd.to_datetime(sospechosos_por_fecha["FECHA_SINTOMAS"], format = "%Y-%m-%d")


In [ ]:
bpl.output_notebook()

In [ ]:
# Ploting each line in different figures ...
def plotCases(abscisa, ordenada, x_title, y_title, g_title, paint ):
    fig_1 = px.line( x = abscisa, y = ordenada, title = g_title, width = 1500, height = 600) # labels parameter can be added with a dict

    fig_1.update_traces(marker = dict(color = paint), line = dict(color = paint, width = 2))
    fig_1.update_xaxes(showgrid = True, title = x_title)
    fig_1.update_yaxes(showgrid = True, title = y_title)
    fig_1.show()
    
plotCases(confirmados_por_fecha["fecha_ingreso"], confirmados_por_fecha["confirmados"], 
        "Fecha de registro", "Casos confirmados", "Casos de COVID-19 en México","#b4004e")
plotCases(defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones"], 
        "Fecha de defunción", "Defunciones", "Defunciones por COVID-19 en México" , "black")
# plotCases(sospechosos_por_fecha["fecha_sintomas"],sospechosos_por_fecha["sospechosos"], 
#         "Fecha del inicio de sintomas", "Casos sospechosos", "Casos sospechosos de COVID-19 en México", "#c67c00")


In [ ]:
# Computing the comulative cases ...
def commulative_cases(data):
    data_vec = np.array(data)
    data_acumulados = np.cumsum(data_vec)    
    return pd.Series(data_acumulados)                           # Numpy array to pandas Serie

# Commulative confirmed cases ....
confirmados_por_fecha["casos_acumulados"] = commulative_cases(confirmados_por_fecha["confirmados"])
# Commulative Deaths  ....
defunciones_por_fecha["defunciones_acumuladas"] = commulative_cases(defunciones_por_fecha["defunciones"])

In [ ]:
#Computing the average with the actual day plus the 6 previous days
def average_of_seven_days(serie , threshold = 25000):
    data_vect = np.array(serie)
    average = np.zeros(data_vect.size)
    counter = 0
    init = 0
    # ....
    index = np.nonzero(data_vect > threshold)
    data_vect[index] = 0
    #Computing the average
    for i in range(data_vect.size):
        if i >= 6:
            average[counter] = np.mean(data_vect[init:i+1])
            init += 1
        else:
            average[counter] = 1/7 * np.sum(data_vect[0:i+1])

        counter += 1

    return  pd.Series(np.round(average))   #Return a pandas Serie with rounded values

In [ ]:
# confirmados_por_fecha["casos_promedio"] =  average_of_seven_days(confirmados_por_fecha["confirmados"])
# defunciones_por_fecha["defunciones_promedio"] = average_of_seven_days(defunciones_por_fecha["defunciones"])

In [ ]:

# fig_bar1_conf = px.bar(x =confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
    
#                     labels= dict(x = "Fecha de registro", y = "Nuevos casos confirmados"), title="Nuevos casos de COVID-19 reportados diariamente", width = 900)
#fig_bar1_conf.update_traces( marker = dict(color = "#ff1744", opacity = 0.6))
# fig_bar2_def = px.bar(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"],
#                         labels = dict(x = "Fecha de defunción", y = "Defunciones"), title = "Nuevas defunciones por COVID-19 reportadas diariamente", width = 900)

# fig_bar2_def.update_traces(marker = dict(color = "black"))

# Bar plots for confirmed and death cases ...
def show_confirmed_cases(x_vect,y_vect, region, anomalia):

    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900

    fig_bar1 = go.Figure()
    fig_bar1.add_trace(go.Bar(x = x_vect, y = y_vect, 
                                marker = dict(color = "#ff1744", opacity = 0.6 ), name = "Nuevos casos"))
    #Average of seven days curve ...                             
    fig_bar1.add_trace(go.Scatter(x = x_vect, y =  average_of_seven_days(y_vect),
                                        mode = "lines", line = dict(color = "#b71c1c", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar1.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar1.update_yaxes(title = "Nuevos casos confirmados")
    fig_bar1.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevos casos de COVID-19 reportados diariamente en " + region)
    #Showing figure ... 
    fig_bar1.show()

def show_deaths(x_vect,y_vect,region, anomalia):
    
    if anomalia == True:
        h = 600
        w = 1000
    else:
        h = 450
        w = 900
    # Death cases ...
    fig_bar2 = go.Figure()
    fig_bar2.add_trace(go.Bar(x = x_vect, y = y_vect,
                                marker = dict(color = "#546e7a", opacity = 0.75), name = "Nuevas defunciones"))
    #Average of seven days curve ...                         
    fig_bar2.add_trace(go.Scatter(x = x_vect, y =average_of_seven_days(y_vect, threshold = 2500),
                                        mode = "lines", line = dict(color = "black", width = 2.5), name = "Promedio de 7 días"))
    #custom layout
    fig_bar2.update_xaxes(title = "Fecha de registro",spikethickness= .4)
    fig_bar2.update_yaxes(title = "Nuevas defunciones confirmadas")
    fig_bar2.update_layout(width = w, height = h, hovermode = "x unified", title = "Nuevas defunciones de COVID-19 reportadas diariamente en " + region)
    #Showing figure ...
    fig_bar2.show()

show_confirmed_cases(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["confirmados"], "México", False)
show_deaths(defunciones_por_fecha["fecha_defuncion"],defunciones_por_fecha["defunciones"],"México", False)

In [ ]:
# Ploting the commulative confirmed and deaths cases ...

def acum_plot(x_conf_data, y_conf_data, x_def_data, y_def_data, region ):
    fig_acum = go.Figure()

    fig_acum.add_trace(go.Scatter(x = x_conf_data, y = y_conf_data,
                    mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

    fig_acum.add_trace(go.Scatter(x = x_def_data, y = y_def_data,
                    mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))

    fig_acum.update_xaxes(title = "Fecha de registro")
    fig_acum.update_yaxes(title = "Número de casos acumulados")
    fig_acum.update_layout(title = "Casos acumulados en " + region )
    fig_acum.show()


# fig_acum = go.Figure()

# fig_acum.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["casos_acumulados"],
#                     mode = "lines" , line = dict(color = "#ff1744", width = 2.5), name = "Casos confirmados" ))

# fig_acum.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones_acumuladas"],
#                     mode = "lines", line = dict(color = "black", width = 2.5), name = "Defunciones "))



# fig_acum.update_xaxes(title = "Fecha de registro")
# fig_acum.update_yaxes(title = "Número de casos acumulados")
# fig_acum.update_layout(title = "Casos acumulados en México")
# fig_acum.show()

acum_plot(confirmados_por_fecha["fecha_ingreso"],confirmados_por_fecha["casos_acumulados"],
            defunciones_por_fecha["fecha_defuncion"], defunciones_por_fecha["defunciones_acumuladas"], region = "México" )


In [ ]:

# Ploting each line in the same figure ...

# fig_2 = px.line(confirmados_por_fecha, x = "fecha_sintomas", y = "confirmados", width = 1000, height= 300,
#                  labels= dict(fecha_sintomas = "Fecha", confirmados = "Número de personas", name = "Casos confirmados"))

fig_2 = go.Figure()

fig_2.add_trace(go.Scatter(x = confirmados_por_fecha["fecha_ingreso"], y = confirmados_por_fecha["confirmados"], 
                mode = "lines",name = "Casos confirmados", line = dict(color = "#b4004e")) )
fig_2.add_trace(go.Scatter(x = defunciones_por_fecha["fecha_defuncion"], y = defunciones_por_fecha["defunciones"], 
                mode = "lines",name = "Defunciones", line = dict(color = "black") ) )
fig_2.add_trace(go.Scatter(x = sospechosos_por_fecha["fecha_sintomas"], y = sospechosos_por_fecha["sospechosos"],
                mode = "lines", name = "Casos sospechosos", line = dict(color = "#c67c00")))

fig_2.update_xaxes(showgrid = True, title = "Fecha")
fig_2.update_yaxes(showgrid = True , title = "Número de personas")
fig_2.update_layout( width = 1200, height= 400 , title = "COVID-19 México")
fig_2.show()

In [ ]:
# Distribution of cases per state ...................................
confirmados

In [ ]:
def time_usage(func):
    def wrapper(*args, **kwargs):
        beg_ts = time.time()
        retval = func(*args, **kwargs)
        end_ts = time.time()
        print("elapsed time: %f" % (end_ts - beg_ts))
        return retval
    return wrapper

In [ ]:
#Grouping by state ENTIDAD_RES
confirmados_por_estado = confirmados.groupby("ENTIDAD_RES").size().to_frame("Casos confirmados").reset_index()
confirmados_por_estado

In [ ]:
# Cargando el diccionario de estados
dics = pd.read_excel("https://github.com/Riloro/mexico_covid19_dashboard/blob/master/diccionarios/201128%20Catalogos_.xlsx?raw=true", sheet_name = None)
dics.keys()

In [ ]:
dic_estados = dics["Catálogo de ENTIDADES"]
dic_estados

In [ ]:
confirmados_por_entidad = pd.merge(confirmados_por_estado, dic_estados, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD" ) # left outer join
confirmados_por_entidad

In [ ]:
confirmados_por_entidad.dtypes

In [ ]:
# Bar plot of the confirmed cases by state
confirmados_por_entidad.sort_values("Casos confirmados", ascending = False, inplace = True ) # Ordenando por número de casos de forma descendente 
fig_bar_edo = px.bar( x = confirmados_por_entidad["ENTIDAD_FEDERATIVA"], y = confirmados_por_entidad["Casos confirmados"],
                        labels = dict(x = "Entidad Federativa", y = "Número de casos confirmados"), title = "Número de casos confirmados por estado")


fig_bar_edo.update_xaxes(type='category', tickfont=dict(size = 10))
fig_bar_edo.update_traces(marker = dict(color = "#b71c1c"), opacity = 0.8)
fig_bar_edo.show()

In [5]:
stime = timeit.default_timer()
poblaciones = pd.read_csv("http://www.conapo.gob.mx/work/models/CONAPO/Datos_Abiertos/Proyecciones2018/pob_mit_proyecciones.csv", encoding="latin1")
e = timeit.default_timer() - stime
poblaciones
print(e)

50.69830970000112


In [10]:
poblacion_2020 = poblaciones[poblaciones["AÑO"] == 2020]
poblacion_2020.to_csv("C:/Users/user/Desktop/poblacion2020_new.csv", encoding='latin1', index = False)

In [9]:
poblacion_2020

,RENGLON,AÑO,ENTIDAD,CVE_GEO,EDAD,SEXO,POBLACION
15400,15401,2020,República Mexicana,0,0,Hombres,1087447
15401,15402,2020,República Mexicana,0,0,Mujeres,1048275
15402,15403,2020,República Mexicana,0,1,Hombres,1093379
15403,15404,2020,República Mexicana,0,1,Mujeres,1054636
15404,15405,2020,República Mexicana,0,2,Hombres,1100800
...,...,...,...,...,...,...,...
585855,585856,2020,Zacatecas,32,107,Mujeres,1
585856,585857,2020,Zacatecas,32,108,Hombres,0
585857,585858,2020,Zacatecas,32,108,Mujeres,0
585858,585859,2020,Zacatecas,32,109,Hombres,0


In [ ]:
grupos_pob = poblacion_2020.groupby(["CVE_GEO","ENTIDAD"])
poblacion_tot_estado = grupos_pob["POBLACION"].sum().to_frame("POBLACION").reset_index()   # Sum operation in the POBLACION column of each group

In [ ]:
datos_por_estado = pd.merge(confirmados_por_entidad, poblacion_tot_estado, how = "left", left_on = "CLAVE_ENTIDAD", right_on = "CVE_GEO" )
datos_por_estado.columns

In [ ]:
poblacion_tot_estado

In [ ]:
lista_estados = list(datos_por_estado["ENTIDAD"])
print(lista_estados)

In [ ]:
# Rate of confirmed case by 100,000 people 
datos_por_estado["TASA DE CASOS POR 100K"] = 100000 * datos_por_estado["Casos confirmados"]/datos_por_estado["POBLACION"]
datos_por_estado.sort_values("TASA DE CASOS POR 100K", ascending = False, inplace = True)
datos_por_estado.reset_index(drop = True, inplace = True)
datos_por_estado

In [ ]:
# Bar plot of the rate of confirmed cases per 100k persons 
fig_bar_cases_rate = px.bar(x = datos_por_estado["ENTIDAD"], y = datos_por_estado["TASA DE CASOS POR 100K"],
                            labels = dict(y = "Casos confirmados / 100k personas" , x = "") ,title = "Tasa de casos confirmados por cada 100,000 personas")


fig_bar_cases_rate.update_traces(marker = dict(color = "#ff616f"))
#fig_bar_cases_rate.update_xaxes(tickfont = dict(size = 11))
fig_bar_cases_rate.show()

In [ ]:
# Grouping by state ENTIDAD_RES and FECHA_INGRESO
confirmados_por_estado_fecha = confirmados.groupby(["ENTIDAD_RES","FECHA_INGRESO"]).size().to_frame("confirmados").reset_index()
confirmados_por_estado_fecha["fecha_ingreso"] = pd.to_datetime(confirmados_por_estado_fecha["FECHA_INGRESO"], format = "%Y-%m-%d")


confirmados_por_estado_fecha

In [ ]:
defunciones_por_estado_fecha = defunciones.groupby(["ENTIDAD_RES", "FECHA_DEF"]).size().to_frame("defunciones").reset_index()
defunciones_por_estado_fecha["fecha_defuncion"] = pd.to_datetime(defunciones_por_estado_fecha["FECHA_DEF"], format = "%Y-%m-%d")
defunciones_por_estado_fecha

In [ ]:
entidad_clave = datos_por_estado["ENTIDAD"].to_frame()
entidad_clave["CLAVE_ENTIDAD"] = datos_por_estado["CLAVE_ENTIDAD"]
# Merge between entidad_cave and confirmados_por_estado_fecha
confirmados_por_estado_fecha_clave = pd.merge(confirmados_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
confirmados_por_estado_fecha_clave

In [ ]:
# Merge betweem entidad_clave and defunciones_por_estado_fecha
defunciones_por_estado_fecha_clave = pd.merge(defunciones_por_estado_fecha, entidad_clave, how = "left", left_on = "ENTIDAD_RES", right_on = "CLAVE_ENTIDAD")
defunciones_por_estado_fecha_clave

In [ ]:
clave_estado = 32
confirmados_mich = confirmados_por_estado_fecha_clave[confirmados_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado]
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())
confirmados_mich.reset_index(drop = True, inplace = True)
confirmados_mich["confirmados_acumulados"] = commulative_cases(confirmados_mich["confirmados"]) # Accumulated confirmed cases in Mich.
confirmados_mich

In [ ]:
defunciones_mich = defunciones_por_estado_fecha_clave[defunciones_por_estado_fecha_clave["ENTIDAD_RES"] == clave_estado ]
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
defunciones_mich.reset_index(drop = True, inplace = True)     
defunciones_mich["defunciones_acumuladas"] = commulative_cases(defunciones_mich["defunciones"])
defunciones_mich

In [ ]:
#
acum_plot(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados_acumulados"],
            defunciones_mich["fecha_defuncion"], defunciones_mich["defunciones_acumuladas"], region = confirmados_mich["ENTIDAD"].iloc[0] )
print("Total de defunciones acumuladas = ", defunciones_mich["defunciones"].sum())
print("Total de casos confirmados = ",confirmados_mich["confirmados"].sum())

In [ ]:
show_confirmed_cases(confirmados_mich["fecha_ingreso"],confirmados_mich["confirmados"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)
show_deaths(defunciones_mich["fecha_defuncion"],defunciones_mich["defunciones"], region = confirmados_mich["ENTIDAD"].iloc[0], anomalia = False)

In [ ]:
jhu_daily_reports = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-17-2021.csv", encoding="utf-8")
mexico_daily_report = jhu_daily_reports[jhu_daily_reports["Country_Region"] == "Mexico"]
mexico_daily_report

In [ ]:

start_time = timeit.default_timer()
#Casos confirmados reportados por JHU ....
jhu_confirmed_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv", encoding = "utf-8")
jhu_confirmed_mexico = jhu_confirmed_time_serie[jhu_confirmed_time_serie["Country/Region"] == "Mexico"]
#Muertes reportadas por JHU ...
jhu_deaths_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv",encoding="utf-8")
jhu_deaths_mexico = jhu_deaths_time_serie[jhu_deaths_time_serie["Country/Region"] == "Mexico"]
#Recuperados reportados por JHU ...
jhu_recovered_time_serie = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv",encoding="utf-8")
jhu_recovered_mexico = jhu_recovered_time_serie[jhu_recovered_time_serie["Country/Region"] == "Mexico"]
elapsed = timeit.default_timer() - start_time
print(elapsed)

In [ ]:
raw_confirmed_data = jhu_confirmed_mexico.reset_index(drop=True).transpose().reset_index()
raw_deats_data = jhu_deaths_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data = jhu_recovered_mexico.reset_index(drop = True).transpose().reset_index()
raw_recovered_data

In [ ]:
#Insert a character in the ith position
def insert_str(init_str, insert_str, i):
    return init_str[:i] + insert_str + init_str[i:]

In [ ]:
def insert_commas(string):
    step = 3
    size = len(string)
    quotient = size//step
    mod = size % step
    
    if mod == 0:
        commas_number = quotient - 1
        init_index = step
    else:
        commas_number = quotient
        init_index = mod 

    for i in range(commas_number):
        string = insert_str(string, ",", init_index )
        init_index = init_index + step + 1
    return string

In [ ]:
#Filtrando el numero de casos y fechas de cada dataFrame ...
jhu_conf_mex_df = raw_confirmed_data["index"].iloc[4:].to_frame("fecha_confirmados").reset_index(drop = True)
jhu_conf_mex_df["confirmados"] = raw_confirmed_data[0].iloc[4:].reset_index(drop = True)
jhu_conf_mex_df["fecha_confirmados_dateTime"] = pd.to_datetime(jhu_conf_mex_df["fecha_confirmados"], format = "%m/%d/%y")

jhu_deaths_mex_df = raw_deats_data["index"].iloc[4:].to_frame("fecha_muerte").reset_index(drop = True)
jhu_deaths_mex_df["muertes"] = raw_deats_data[0].iloc[4:].reset_index(drop = True)
jhu_deaths_mex_df["fecha_muerte_dateTime"] = pd.to_datetime(jhu_deaths_mex_df["fecha_muerte"], format = "%m/%d/%y")

jhu_recovered_mex_df = raw_recovered_data["index"].iloc[4:].to_frame("fecha_recuperado").reset_index(drop = True)
jhu_recovered_mex_df["recuperados"] = raw_recovered_data[0].iloc[4:].reset_index(drop = True)
jhu_recovered_mex_df["feha_recuperados_dateTime"] = pd.to_datetime(jhu_recovered_mex_df["fecha_recuperado"], format = "%m/%d/%y")

In [ ]:
# Inserting commas ...
str_conf_mex = str(jhu_conf_mex_df["confirmados"].iloc[-1])
str_deaths_mex = str(jhu_deaths_mex_df["muertes"].iloc[-1])
str_recovered_mex = str(jhu_recovered_mex_df["recuperados"].iloc[-1])
#Calling inset_commas function ...
str_conf_mex_commas = insert_commas(str_conf_mex)
str_recovered_mex_commas = insert_commas(str_recovered_mex)
str_deaths_mex_commas = insert_commas(str_deaths_mex)
print(str_conf_mex_commas)
print(str_recovered_mex_commas)
print(str_deaths_mex_commas)

In [ ]:
jhu_deaths_mex_df

In [ ]:
#Graficando los casos confirmados de muertes y confirmados ....
acum_plot(jhu_conf_mex_df["fecha_confirmados_dateTime"], jhu_conf_mex_df["confirmados"],
            jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes"], region="México")

In [ ]:
# difference JHU data in order to get daily cases ....
def diff(data):
    v_1 = np.array(data)
    v_2 = np.diff(v_1)
    res = np.insert(v_2,0,v_1[0])

    return pd.Series(res)

In [ ]:
#Getting daily cases ...
jhu_conf_daily = diff(jhu_conf_mex_df["confirmados"])   
jhu_conf_mex_df["casos diarios"] = jhu_conf_daily

#Getting daily deaths...
jhu_daily_deaths = diff(jhu_deaths_mex_df["muertes"]) 
jhu_deaths_mex_df["muertes diarias"] = jhu_daily_deaths

jhu_deaths_mex_df

In [ ]:
jhu_conf_mex_df

In [ ]:
 
 show_confirmed_cases(jhu_conf_mex_df["fecha_confirmados_dateTime"],jhu_conf_mex_df["casos diarios"], region = "México", anomalia = True)
 show_deaths(jhu_deaths_mex_df["fecha_muerte_dateTime"], jhu_deaths_mex_df["muertes diarias"],  region = "México", anomalia = True)

In [ ]:
confirmados_por_estado_fecha_clave